> In many applications, data may be spread across a number of files or databases or be arranged in a form that is not easy to analyze. This chapter focuses on tools to help combine, join, and rearrange data

# 8.1 Hierarchical Indexing

Hierarchical indexing is an important feature of pandas that enables you to **have multiple (two or more) index levels on an axis**

Somewhat abstractly, it **provides a way for you to work with higher dimensional data in a lower dimensional form**

Let’s start with a simple example; create a Series with a list of lists (or arrays) as the index:

In [68]:
import pandas as pd
import numpy as np

In [69]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [70]:
data

a  1   -0.919762
   2   -0.726083
   3   -0.536927
b  1    0.415066
   3    0.950097
c  1    0.793184
   2   -0.445442
d  2   -0.421842
   3    0.592828
dtype: float64

What you’re seeing is a prettified view of a Series with a MultiIndex as its index

The “gaps” in the index display mean “use the label directly above”:

In [71]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

With a hierarchically indexed object, so-called **partial indexing** is possible, enabling you to concisely select subsets of the data:

In [72]:
data['b']

1    0.415066
3    0.950097
dtype: float64

In [73]:
data['b':'c']

b  1    0.415066
   3    0.950097
c  1    0.793184
   2   -0.445442
dtype: float64

In [74]:
data.loc[['b', 'd']]

b  1    0.415066
   3    0.950097
d  2   -0.421842
   3    0.592828
dtype: float64

Selection is even possible from an “inner” level:

In [75]:
data.loc[:,2]

a   -0.726083
c   -0.445442
d   -0.421842
dtype: float64

Hierarchical indexing plays an important role in reshaping data and group-based operations like forming a pivot table

For example, **you could rearrange the data into a DataFrame using its unstack method:**

In [76]:
data.unstack()

,1,2,3
a,-0.919762,-0.726083,-0.536927
b,0.415066,NaN,0.950097
c,0.793184,-0.445442,NaN
d,NaN,-0.421842,0.592828


For example, you could rearrange the data into a DataFrame using its unstack method:

In [77]:
data.unstack().stack()

a  1   -0.919762
   2   -0.726083
   3   -0.536927
b  1    0.415066
   3    0.950097
c  1    0.793184
   2   -0.445442
d  2   -0.421842
   3    0.592828
dtype: float64

With a **DataFrame, either axis can have a hierarchical index:**

In [78]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])

In [79]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

The hierarchical levels can have names (as strings or any Python objects). If so, these will show up in the console output:

In [80]:
frame.index.names = ['key1', 'key2']

In [81]:
frame.columns.names = ['state', 'color']

With partial column indexing you can similarly select groups of columns:

In [85]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [86]:
frame['Ohio', 'Green']

key1  key2
a     1       0
      2       3
b     1       6
      2       9
Name: (Ohio, Green), dtype: int64

A MultiIndex can be created by itself and then reused; the columns in the preceding DataFrame with level names could be created like this:

In [83]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

## Reordering and Sorting Levels

At times you will need to **rearrange the order of the levels on an axis or sort the data by the values in one specific level**

The **swaplevel takes two level numbers or names and returns a new object with the levels interchanged **(but the data is otherwise unaltered):

In [84]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

**sort_index, on the other hand, sorts the data using only the values in a single level**

When swapping levels, it’s not uncommon to also use sort_index so that the result is lexicographically sorted by the indicated level:

In [91]:
frame.sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [92]:
frame.sort_index(level=1) # Sort By key2

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [96]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## Summary Statistics by Level

Many descriptive and summary statistics on DataFrame and Series have a **level option in which you can specify the level you want to aggregate by on a particular axis**

Consider the above DataFrame; we can aggregate by level on either the rows or columns like so:

In [101]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [104]:
frame.sum(level='color', axis=1) # Ohio Green + Colorado Green

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

Under the hood, this utilizes pandas’s groupby machinery

## Indexing with a DataFrame’s columns

It’s not unusual to want to use one or more columns from a DataFrame as the row index

Alternatively, **you may wish to move the row index into the DataFrame’s columns.** Here’s an example DataFrame:

In [106]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})

In [107]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3
